<a href="https://colab.research.google.com/github/linxizhu1209/AI-Repo/blob/main/RAG_%E1%84%80%E1%85%B5%E1%84%87%E1%85%A1%E1%86%AB_%E1%84%86%E1%85%AE%E1%86%AB%E1%84%89%E1%85%A5_%E1%84%8C%E1%85%B5%E1%86%AF%E1%84%8B%E1%85%B4%E1%84%8B%E1%85%B3%E1%86%BC%E1%84%83%E1%85%A1%E1%86%B8_%E1%84%89%E1%85%B5%E1%84%89%E1%85%B3%E1%84%90%E1%85%A6%E1%86%B7_%E1%84%80%E1%85%AE%E1%84%8E%E1%85%AE%E1%86%A8_%E1%84%91%E1%85%B3%E1%84%85%E1%85%A9%E1%84%8C%E1%85%A6%E1%86%A8%E1%84%90%E1%85%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install PyPDF2 faiss-cpu numpy sentence-transformers nltk openai

In [37]:
import os
import glob
import warnings
from typing import List

import PyPDF2
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer, util
from nltk.tokenize import sent_tokenize

from openai import OpenAI

warnings.filterwarnings('ignore')


## PDF 파싱


In [38]:
def extract_text_pypdf2(pdf_path):
    """PyPDF2를 사용하여 PDF에서 텍스트 추출"""

    text = ""
    ########################################
    #### 문제 1. PDF 텍스트 추출

    try:
      with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        print(f"총 페이지 수: {len(pdf_reader.pages)}")

        for page_num in range(len(pdf_reader.pages)):
          page = pdf_reader.pages[page_num]
          text += page.extract_text()

    except Exception as e:
        print(f"오류 발생: {e}")
        return None
    ########################################

    return text


In [39]:
pdf_folder = "data/"
pdf_files = glob.glob(os.path.join(pdf_folder, "*.pdf"))

for pdf_file in pdf_files:
    print(f"- {os.path.basename(pdf_file)}")


- Peak China론을 통해 본 미중 패권 경쟁 방향성_게시용.pdf
- 중국 부동산 신용 리스크 점검_게시용.pdf
- 연자24-01_2024년 국제 원유 시황과 유가 전망_웹용.pdf
- KB자영업 분석 보고서3_커피전문점 현황 및 시장여건 분석.pdf


In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
all_texts = []
document_sources = []

for pdf_file in pdf_files:
    text = extract_text_pypdf2(pdf_file)
    if text:
        all_texts.append(text+"\n")
        document_sources.append(os.path.basename(pdf_file))
        print(f"추출된 텍스트 길이: {len(text)} 문자\n")

총 페이지 수: 17
추출된 텍스트 길이: 14758 문자

총 페이지 수: 18
추출된 텍스트 길이: 14910 문자

총 페이지 수: 33
추출된 텍스트 길이: 66135 문자

총 페이지 수: 23
추출된 텍스트 길이: 22117 문자



## 의미기반 청킹


In [42]:
import nltk
nltk.download('punkt_tab')

## 문장을 어떻게 나눌지에 대한 규칙(모델/데이터) 다운로드

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [43]:

def semantic_chunker(text: str, model: SentenceTransformer, similarity_threshold: float = 0.85) -> List[str]:
    """
    의미 기반으로 텍스트를 청크로 분할
    - 각 문장 embedding들끼리 유사도를 계산하여 유사도가 높은 문장들을 하나의 청크로 묶음
    """
    sentences = sent_tokenize(text) # 텍스트를 문장 단위 리스트로 잘라주는 함수

    if len(sentences) == 0:
        return []

    if len(sentences) == 1:
        return sentences

    embeddings = model.encode(sentences, convert_to_tensor=True)

    chunks = []
    current_chunk = [sentences[0]]

    for i in range(1, len(sentences)):
        similarity = util.pytorch_cos_sim(embeddings[i - 1], embeddings[i]).item()
        if similarity > similarity_threshold:
            current_chunk.append(sentences[i]) # 이전 문장과 현 문장이 기준 유사도 넘는 경우 같은 청크로 묶음
        else:
            # 유사도가 기준을 넘지 못한 경우, 이전 문장은 다른 청크로 분리되고, 현 문장은 하나의 청크로 생성
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentences[i]]

    # for문을 다 돌고나서 남은 chunk 처리
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks


NLTK -> 텍스트처리 및 토큰화 라이브러리
sent_tokenize(text) -> 텍스트를 문장 단위 리스트로 잘라줌


SentenceTransformer(임베딩 라이브러리) -> model.encode(sentences) -> 문장을 벡터(숫자 표현)로 변환해주는 함수

utl.pythorch_cos_sim -> 숫자끼리 유사도 계산해주는 함수

In [44]:
chunking_model = SentenceTransformer('snumin44/simcse-ko-roberta-unsupervised')

## 특정 모델 -> 문장의 의미를 숫자 벡터로 표현하도록 학습된 모델


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

In [45]:
all_chunks = []
chunk_sources = []

for idx, text in enumerate(all_texts):
    print(f"\n문서 {idx+1}/{len(all_texts)} 청킹 중: {document_sources[idx]}")
    chunks = semantic_chunker(text, chunking_model, similarity_threshold=0.80)
    print(f"생성된 청크 수: {len(chunks)}")

    all_chunks.extend(chunks)
    chunk_sources.extend([document_sources[idx]] * len(chunks))

print(f"\n총 청크 수: {len(all_chunks)}")



문서 1/4 청킹 중: Peak China론을 통해 본 미중 패권 경쟁 방향성_게시용.pdf
생성된 청크 수: 27

문서 2/4 청킹 중: 중국 부동산 신용 리스크 점검_게시용.pdf
생성된 청크 수: 41

문서 3/4 청킹 중: 연자24-01_2024년 국제 원유 시황과 유가 전망_웹용.pdf
생성된 청크 수: 390

문서 4/4 청킹 중: KB자영업 분석 보고서3_커피전문점 현황 및 시장여건 분석.pdf
생성된 청크 수: 11

총 청크 수: 469


In [70]:
for i in range(3):
    print(f"\n청크 {i+1} (출처: {chunk_sources[i]}):")
    print(f"{all_chunks[i][:100]}...")



청크 1 (출처: Peak China론을 통해 본 미중 패권 경쟁 방향성_게시용.pdf):
 
 
 
 
 
 
  
 
 
 
 
 
 
2023....

청크 2 (출처: Peak China론을 통해 본 미중 패권 경쟁 방향성_게시용.pdf):
05 
이정진  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
            
 
 
 
 
  1....

청크 3 (출처: Peak China론을 통해 본 미중 패권 경쟁 방향성_게시용.pdf):
중국의  성장은  정점에 도달(Peak China) 하였나 ?...


## FAISS 벡터 DB 구축


In [47]:
embedding_model = SentenceTransformer('snumin44/simcse-ko-roberta-unsupervised')

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

In [73]:
#### 문제 2. 청크 임베딩 & FAISS 인덱스 구축

sentence_embeddings = embedding_model.encode(all_chunks)

dim = sentence_embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.is_trained

index.add(sentence_embeddings)

print(f"FAISS 인덱스에 추가된 벡터 수: {index.ntotal}")
sentence_embeddings.shape

FAISS 인덱스에 추가된 벡터 수: 469


(469, 768)

sentence_embeddings.shape -> (문장 수, 임베딩 차원) -> 임베딩 차원은 모델마다 다르며, Bert계열은 768 사용

IndexFlat2는 학습이 필요없는 인덱스임.
들어온 벡터를 그대로 저장하고 거리 계산만 하는 단순한 구조임

faiss.IndexFlat2(sentece_embeddings.shape[1]) -> 앞으로 들어올 모든 벡터는 길이가 sentence_embeddings.shape[1] (dim) 이어야한다는 것

embedding_mode.encode와 같이 embedding과정을 semantic_chunker 함수 및 위에서 2번한 이유 -> semantic_chunker 함수에서는 similarity를 비교해 청크로 묶기위함, 위 코드는 index에 추가하기 위함

## 유사 문서 검색


In [74]:
def search_similar_chunks(query: str, k: int = 3):
    """쿼리와 유사한 청크 검색"""

    ########################################
    #### 문제 3. 질의-유사 청크 검색 함수 작성

    # query를 리스트로 넣어야함.
    # FAISS는 query를 (nq, dim) 형태로 받기에 그냥 query(dim, ) 형태가 아닌 (1,dim)으로 바꿔줘야함
    xq = embedding_model.encode([query])

    distances, indices = index.search(xq, k)

    ########################################

    results = []
    for i, idx in enumerate(indices[0]):
        results.append({
            'rank': i + 1,
            'chunk': all_chunks[idx],
            'source': chunk_sources[idx],
            'distance': distances[0][i]
        })


    return results


In [75]:
test_query = "중국 경제의 현재 상황은?"
results = search_similar_chunks(test_query, k=3)

print(f"질문: {test_query}\n")
for result in results:
    print(f"순위 {result['rank']} (거리: {result['distance']:.4f})")
    print(f"출처: {result['source']}")
    print(f"내용: {result['chunk'][:100]}...\n")
    print("-"*50)


질문: 중국 경제의 현재 상황은?

순위 1 (거리: 152.8885)
출처: Peak China론을 통해 본 미중 패권 경쟁 방향성_게시용.pdf
내용: 높을수록  생산성  
저하를  의미 
자료: World Bank  
 (1인 독재) 시진핑  친정 체제 구축 등 전체주의  강화와  이에 따른 지방정부와  민영부문의  활
력 저...

--------------------------------------------------
순위 2 (거리: 157.3765)
출처: 연자24-01_2024년 국제 원유 시황과 유가 전망_웹용.pdf
내용: • 중국의 성장률은 2023년 5.2%에서 2024년 4.2%로 하락할 것으로 전망했는데, 부동산 위기로 인한 투자 
  위축과 소비심리 제약, 높은 청년실업률 등의 취약성이 지적...

--------------------------------------------------
순위 3 (거리: 158.7196)
출처: 중국 부동산 신용 리스크 점검_게시용.pdf
내용: 부동산  경기 침체의  거시경제  영향은 ?...

--------------------------------------------------


## OpenAI API를 활용한 답변 생성


In [56]:
from google.colab import userdata

In [59]:
OPENAI_API_KEY = userdata.get('OPEN_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)


In [76]:
# chunk와 source 포맷팅 함수
def build_context(search_results):
  context_blocks = []

  for result in search_results:
    block = (
        f"Source: {result['source']}\n"
        f"Content:\n{result['chunk']}\n"
    )
    context_blocks.append(block)

  return "\n---\n".join(context_blocks)

# gpt 메시지 구성
def build_messages(query, context):
  system_prompt = (
      "You are a helpful assistant. \n"
      "Answer ONLY using the provided context. \n"
      "If the answer is not in the context, say you don't know. \n"
      "Cite the source number like 1., 2."
  )

  user_prompt = (
      f"Question: {query}\n"
      f"Context: {context}\n"
  )

  return [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_prompt},
  ]

## 검색된 문서를 기반으로 답변 생성
def generate_answer(query: str, k: int = 3):
    ########################################
    #### 문제 4. 검색 컨텍스트 기반 LLM 응답 생성

    search_results = search_similar_chunks(query, k)

    context = build_context(search_results)

    messages = build_messages(query, context)

    ## GPT 호출
    response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=messages,
      temperature=0.2,
    )

    return {
        'answer': response.choices[0].message.content,
        'sources': search_results
    }



In [78]:
query = "중국 경제의 현재 상황은?"

result = generate_answer(query, k=3)

print(f"질문: {query}\n")
print("="*80)
print(f"답변:\n{result['answer']}")
print("="*80)
print("\n참고 문서:")
for i, source in enumerate(result['sources']):
    print(f"{i+1}. {source['source']} (유사도: {1/(1+source['distance']):.4f})")


질문: 중국 경제의 현재 상황은?

답변:
중국 경제의 현재 상황은 여러 가지 취약성이 지적되고 있습니다. 2023년 성장률은 5.2%에서 2024년에는 4.2%로 하락할 것으로 전망되며, 이는 부동산 위기로 인한 투자 위축, 소비심리 제약, 높은 청년실업률 등이 원인으로 지적됩니다. 또한, 시진핑의 1인 독재 체제 구축과 이에 따른 지방정부 및 민영부문의 활력 저하가 지속가능한 경제 성장을 제약하고 있습니다. 부동산에 대한 높은 의존도는 경제 성장 구조의 취약함을 드러내고 있습니다. (2., 1.)

참고 문서:
1. Peak China론을 통해 본 미중 패권 경쟁 방향성_게시용.pdf (유사도: 0.0065)
2. 연자24-01_2024년 국제 원유 시황과 유가 전망_웹용.pdf (유사도: 0.0063)
3. 중국 부동산 신용 리스크 점검_게시용.pdf (유사도: 0.0063)


distance (유사도) 는 유클리드 거리를 의미해서, 커질수록 유사하지 않은 것.

In [79]:
query = "커피 전문점 시장의 특징은 무엇인가요?"

result = generate_answer(query, k=3)

print(f"질문: {query}\n")
print("="*80)
print(f"답변:\n{result['answer']}")
print("="*80)
print("\n참고 문서:")
for i, source in enumerate(result['sources']):
    print(f"{i+1}. {source['source']} (유사도: {1/(1+source['distance']):.4f})")


질문: 커피 전문점 시장의 특징은 무엇인가요?

답변:
커피 전문점 시장의 특징은 다음과 같습니다:

1. 커피 소비가 대중화되면서, 성인 1인당 커피 소비량이 2018년 기준 연간 353잔으로 세계 평균을 크게 상회합니다.
2. 2019년 7월 기준으로 전국에 7.1만개의 커피 전문점이 있으며, 전체 매장의 41.2%가 서울과 경기 지역에 집중되어 있습니다.
3. 커피 전문점의 창업은 2018년에 1.4만개로, 폐업은 9천개 수준으로 창업이 폐업보다 많습니다.
4. 커피 전문점은 단독 운영 매장 비중이 높고, 영업시간이 길며, 영업이익률이 음식점보다 높은 경향이 있습니다.
5. 커피에 대한 선호는 30~40대에서 높으며, 소비자의 절반 이상이 습관적으로 커피를 마십니다.
6. 커피 전문점의 선택 요인은 커피의 맛과 접근성입니다.
7. 향후 커피 전문점에 대한 수요는 지속 증가할 것으로 예상되며, 저가 시장과 스페셜티 고가 시장이 확대될 것입니다. 그러나 매장 수의 빠른 증가로 경쟁이 심화되고 있습니다. 창업 시 상권과 입지, 수요에 대한 면밀한 분석이 필요합니다. (Source: 2.)

참고 문서:
1. KB자영업 분석 보고서3_커피전문점 현황 및 시장여건 분석.pdf (유사도: 0.0080)
2. KB자영업 분석 보고서3_커피전문점 현황 및 시장여건 분석.pdf (유사도: 0.0075)
3. KB자영업 분석 보고서3_커피전문점 현황 및 시장여건 분석.pdf (유사도: 0.0063)
